In [216]:
# import libraries
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
from tqdm import tqdm

In [217]:
# setting up chrome options
chrome_options = Options()
# browsing in incognito mode
chrome_options.add_argument("--incognito")
# setting window size to 1920x1080
chrome_options.add_argument("--window-size=1920x1080")

In [218]:
# create instance
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=
"/Users/marthab/Desktop/Python_Labs/vinted_crawler/chromedriver")

<ipython-input-218-2d554e9c4169>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=


In [219]:
# set url to Women > Jacquemus
url = "https://www.vinted.de/vetements?brand_id[]=168278&catalog[]=1904"
driver.get(url)

# wait 2 sec for page to fully load
time.sleep(2)

In [220]:
# don't accept cookies
try:
# privatsphäre-einstellung alle ablehnen
    reject_all_button = driver.find_element_by_css_selector("#onetrust-reject-all-handler")
    ActionChains(driver).click(reject_all_button).perform()
#NoSuchElementException thrown if not present
except NoSuchElementException:
    print("Not asked for privacy settings")

# choose country Germany
try:
    germany_button = driver.find_element_by_xpath("/html/body/div[13]/div/div/div/div[3]/div[3]/div[2]/div/h2/span")
    ActionChains(driver).click(germany_button).perform()
except NoSuchElementException:
    print("Not asked for country")

# wait 2 sec for page to fully load
time.sleep(2)

In [221]:
# make sure you are on the right page
if 'brand_id[]=168278&catalog[]=1904' not in driver.current_url:
    url = "https://www.vinted.de/vetements?brand_id[]=168278&catalog[]=1904"
    driver.get(url)
    time.sleep(2)

In [222]:
# get url of all products on first page 
products = driver.find_elements_by_css_selector("a.ItemBox_overlay__1kNfX")
product_urls = [prod.get_attribute("href") for prod in products]

In [224]:
#scraping individual product details
alldetails=[]


for product_url in tqdm(product_urls):
    driver.get(product_url)
    #time.sleep(3)

    # get product id from url
    product_id = product_url.split('/')[-1]

    # get product subcatgory from url
    product_subcat = product_url.split('/')[-2]

    # get product catgory from url
    product_cat = product_url.split('/')[-3]

    # get brand
    brand = driver.find_element_by_xpath("//a[@itemprop='url']/span[@itemprop='name']").text

    # get price
    price = driver.find_element_by_xpath('/html/body/div[5]/div/section/div/div[2]/main/aside/div[1]/div[1]/div[1]/div[1]/span/div').text

    # get size 
    # TO-DO: fix 
    size = driver.find_element_by_xpath('/html/body/div[5]/div/section/div/div[2]/main/aside/div[1]/div[1]/div[2]/div[2]/div[2]').text

    # get item condition
    condition = driver.find_element_by_xpath("//div[@itemprop='itemCondition']").text

    # get colour
    colour = driver.find_element_by_xpath("//div[@itemprop='color']").text

        
    infos ={
        'Product_Id':product_id,
        'Category':product_cat,
        'Sub_Category':product_subcat,
        'Brand':brand,
        'Price':price,
        'Size':size,
        'Condition':condition,
        'URL':product_url}

    alldetails.append(infos)

    

100%|██████████| 22/22 [00:40<00:00,  1.85s/it]


In [151]:
pd.DataFrame(alldetails)

,Product_Id,Category,Sub_Category,Brand,Price,Size,Condition,URL
0,1264815455-jacquemus-la-riviera-kleid,kleider,minikleider,JACQUEMUS,"160,00 €",XS / 34 / 6,GUT,https://www.vinted.de/damen/kleidung/kleider/m...
1,1255929602-jacquemus-la-robe-bahia-minidress,kleider,minikleider,JACQUEMUS,"319,00 €",S / 36 / 8,"NEU, MIT ETIKETT",https://www.vinted.de/damen/kleidung/kleider/m...
2,1236664083-jacquemus-la-robe-saudade-navy-klei...,fur-anlasse,party-and-cocktail,JACQUEMUS,"350,00 €",S / 36 / 8,"NEU, MIT ETIKETT",https://www.vinted.de/damen/kleidung/kleider/f...
3,1264854208-jaquemus-le-gandjo-bag,taschen,handtaschen,JACQUEMUS,"210,00 €","NEU, MIT ETIKETT","NEU, MIT ETIKETT",https://www.vinted.de/damen/taschen/handtasche...
4,1264751001-jacquemus-le-bambino-bag,taschen,handtaschen,JACQUEMUS,"380,00 €","NEU, MIT ETIKETT","NEU, MIT ETIKETT",https://www.vinted.de/damen/taschen/handtasche...
5,1264721676-jacquemus-le-chiquito-moyen-bag,taschen,handtaschen,JACQUEMUS,"390,00 €","NEU, MIT ETIKETT","NEU, MIT ETIKETT",https://www.vinted.de/damen/taschen/handtasche...
6,1264609791-jacquemus-handtasche,taschen,handtaschen,JACQUEMUS,"175,00 €","NEU, MIT ETIKETT","NEU, MIT ETIKETT",https://www.vinted.de/damen/taschen/handtasche...
7,1264055260-jacquemus-le-carre,taschen-and-rucksacke,umhangetaschen,JACQUEMUS,"500,00 €",SEHR GUT,SEHR GUT,https://www.vinted.de/herren/accessoires/tasch...
8,1263716785-jacquemus-ciquito-tasche,taschen,handtaschen,JACQUEMUS,"330,00 €","NEU, MIT ETIKETT","NEU, MIT ETIKETT",https://www.vinted.de/damen/taschen/handtasche...
9,1263680243-jacquemus-tasche,taschen,handtaschen,JACQUEMUS,"250,00 €",NEU,NEU,https://www.vinted.de/damen/taschen/handtasche...
